In [56]:
from openai import OpenAI
import logging

In [3]:
client_gemma = OpenAI(
    base_url="http://pomelk1n-dev.su:8000/v1",
    api_key="token-abc123",
)

print(client_gemma.models.list())

client_qwen = OpenAI(
    base_url="http://pomelk1n-dev.su:8001/v1",
    api_key="token-abc123",
)

print(client_qwen.models.list())

SyncPage[Model](data=[Model(id='google/gemma-2-9b-it', created=1726325750, object='model', owned_by='vllm', root='google/gemma-2-9b-it', parent=None, max_model_len=4096, permission=[{'id': 'modelperm-e8a0816e7ded488f86559be245591d78', 'object': 'model_permission', 'created': 1726325750, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}])], object='list')
SyncPage[Model](data=[Model(id='Qwen/Qwen2-7B-Instruct', created=1726325750, object='model', owned_by='vllm', root='Qwen/Qwen2-7B-Instruct', parent=None, max_model_len=32768, permission=[{'id': 'modelperm-5220b7921e834388a9cc5c9025cfd975', 'object': 'model_permission', 'created': 1726325750, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group'

In [4]:
completion = client_gemma.chat.completions.create(
  model="google/gemma-2-9b-it",
  messages=[
    {"role": "user", "content": "Ты кто?"}
  ]
)

completion.choices[0].message.content

'Я — Gemma, большая языковая модель, обученная компанией Google DeepMind.\n\nЯ могу генерировать текст, переводить языки, писать разные виды творческого контента и отвечать на ваши вопросы в информативном стиле.\n\nЯ все еще нахожусь в разработке, но я многому научилась и с удовольствием поделюсь своими знаниями с вами!\n'

In [5]:
def qwen_generate(system_message, user_message) -> str:
  completion = client_qwen.chat.completions.create(
    model="Qwen/Qwen2-7B-Instruct",
    messages=[
      {"role": "system", "content": system_message},
      {"role": "user", "content": user_message}
    ],
    max_tokens=1024,
    temperature=0.0
  )

  return completion.choices[0].message.content

In [6]:
system_prompt = "Ты - квалифицированный специалист по составлению договоров и дополнительных соглашений к ним."

In [7]:
prompt_1 = """Напиши изменения к договору согласно пожеланиям заказчика. 

Найди в тексте договора только те пункты, которые нужно изменить в соответствии с пожеланиями. Измени текст.

Ответь в формате:
"Номер пункта 
Оригинальный текст -> новый текст
...
"

ПОЖЕЛАНИЯ ЗАКАЗЧИКА:
{0}

ТЕКСТ ДОГОВОРА:
{1}

ИЗМЕНЕНИЯ К ДОГОВОРУ:
"""

In [8]:
with open('../documents/txts/Контракт 45-6.24.doc.txt', 'r') as file:
    content = file.read()

In [9]:
instruction = "Сделай аванс 10% и общую цену контракта 10000р. Также поменяй, что в случае изменения реквизитов срок уведомления составляет 10 дней."

In [10]:
qwen_generation = qwen_generate(system_message=system_prompt, user_message=prompt_1.format(instruction, content))
print(qwen_generation)

"Номер пункта 
Оригинальный текст -> новый текст
...

1.1.  Срок, место поставки установлены в Спецификации (Приложение 2 к настоящему Контракту).
-> Срок, место и сумма поставки установлены в Спецификации (Приложение 2 к настоящему Контракту). Аванс составляет 10% от общего контрактного объема, который составляет 10 000 рублей. 

1.5.  Цена Контракта составляет 7 670 (Семь тысяч шестьсот семьдесят) рублей 00 копеек, в том числе НДС 20%.
-> Цена Контракта составляет 10 000 (Десять тысяч) рублей 00 копеек, в том числе НДС 20%.

1.9.  Аванс составляет 0 % от Цены Контракта.
-> Аванс составляет 10% от общего контрактного объема, который составляет 10 000 рублей.

1.10.  Источник финансирования: Бюджет Пермского края
-> Источник финансирования: Бюджет Пермского края, аванс составляет 10% от общего контрактного объема, который составляет 10 000 рублей.

2.1. Приемка Товара осуществляется в соответствии с Порядком приемки Товара (Приложение №3 к настоящему контракту)
-> Приемка Товара осущес

In [11]:
def gemma_generate(user_message, max_tokens=1024) -> str:
  completion = client_gemma.chat.completions.create(
    model="google/gemma-2-9b-it",
    messages=[
      {"role": "user", "content": user_message}
    ],
    max_tokens=max_tokens,
    temperature=0.0
  )

  return completion.choices[0].message.content

In [12]:
prompt_2 = """Ты - профессиональный составитель документов.

Тебе поступил заказ по формированию изменений в документе.

Система автоматически сформировала изменения в формате. В них могут быть ошибки или лишние изменения.

Твоя задача - выбрать из изменений только те, которые удовлетворяют заказу, и оформить их по шаблону.

Шаблон:
"1.1 Пункт N.M статьи N
«N.M Измененный текст пункта»
1.2 Пункт K.L статьи K
«K.L Измененный текст пункта»
...
"

(!) Соблюдай формат нумерации изменений: 1.1, 1.2 и так далее. Он НЕ СОВПАДАЕТ с номерами пунктов документа.

ЗАКАЗ:
{0}

АВТОМАТИЧЕСКИЕ ИЗМЕНЕНИЯ:
{1}

Ответь строго по шаблону:
"""

In [150]:
gemma_generation = gemma_generate(user_message=prompt_2.format(instruction, qwen_generation))
print(gemma_generation)

1.1 Пункт 1.1 статьи 1
«Срок, место и сумма поставки установлены в Спецификации (Приложение 2 к настоящему Контракту). Аванс составляет 10% от общего контрактного объема, который составляет 10 000 рублей.»
1.2 Пункт 1.5 статьи 1
«Цена Контракта составляет 10 000 (Десять тысяч) рублей 00 копеек, в том числе НДС 20%.»
1.3 Пункт 1.9 статьи 1
«Аванс составляет 10% от общего контрактного объема, который составляет 10 000 рублей.»
1.4 Пункт 2.8 статьи 2
«В случае изменения местонахождения и (или) банковских реквизитов Сторона, у которой произошли такие изменения, обязана в течение 10 (Десяти) рабочих дней с момента вышеуказанных изменений письменно уведомить об этом другую Сторону.» 





In [151]:
entity_prompt = """Твоя задача - найти в отрывках документа эти сущности:

Номер контракта/договора
Населенный пункт
Сторона 1 (заказчик)
ФИО лица Стороны 1
Должность лица Стороны 1
Сторона 2 (исполнитель)
ФИО лица Стороны 2
Должность лица Стороны 2

ОТРЫВОК ДОКУМЕНТА:
{0}

Напиши все сущности в этом порядке без форматирования через новую строку. Если сущности нет в тексте, оставь прочерк (символ '-') в этой строке. Если в тексте есть полные ФИО, пиши их, а не инициалы.
"""

In [152]:
entities_str = gemma_generate(user_message=entity_prompt.format(content[:2000] + content[-500:]))
print(entities_str)

45-6/24
д.Кондратово
Государственное казенное учреждение Пермского края "Управление государственной противопожарной службы Пермского края"
Петухова Диана Анатольевна
руководитель контрактной службы
Общество с ограниченной ответственностью «Автоэксклюзив»
Горелов Михаил Геннадьевич
директор 



In [153]:
entities = entities_str.split('\n')
entities

['45-6/24',
 'д.Кондратово',
 'Государственное казенное учреждение Пермского края "Управление государственной противопожарной службы Пермского края"',
 'Петухова Диана Анатольевна',
 'руководитель контрактной службы',
 'Общество с ограниченной ответственностью «Автоэксклюзив»',
 'Горелов Михаил Геннадьевич',
 'директор ',
 '']

In [154]:
entities[2]

'Государственное казенное учреждение Пермского края "Управление государственной противопожарной службы Пермского края"'

In [155]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

def genitive(word):
    word = word.strip()
    try:
        if word != '-': 
            genitive_word = ''
            for subword in word.split(' '):
                parsed_word = morph.parse(subword)[0]
                genitive_word += parsed_word.inflect({'gent'}).word.capitalize() + ' '
            return genitive_word.strip()
    except Exception as e:
        logging.error(f"{e}: {word}")
        return word
    
    return word

In [156]:
try:
    entities = entities_str.split('\n')
    number = entities[0]
    place = entities[1]
    party_1 = entities[2]
    name_1 = entities[3]
    position_1 = entities[4]
    party_2 = entities[5]
    name_2 = entities[6]
    position_2 = entities[7]
except Exception as e:
    logging.error(e)
    number = '-'
    place = '-'
    party_1 = '-'
    name_1 = '-'
    position_1 = '-'
    party_2 = '-'
    name_2 = '-'
    position_2 = '-'    

In [157]:
from datetime import datetime

# Create a datetime object
now = datetime.now()

# Format the datetime object to string in DD.MM.YY format
timestamp = now.strftime("%d.%m.%y")
print(timestamp)

14.09.24


In [158]:
genitive('-')

'-'

In [159]:
position_2

'директор '

In [160]:
genitive(position_2)

'Директора'

In [161]:
from datetime import datetime

# Create a datetime object
now = datetime.now()

# Format the datetime object to string in DD.MM.YYYY format
timestamp = now.strftime("%d.%m.%Y")

In [162]:
genitive(party_2)

ERROR:root:'NoneType' object has no attribute 'word': Общество с ограниченной ответственностью «Автоэксклюзив»


'Общество с ограниченной ответственностью «Автоэксклюзив»'

In [163]:
template = f"""<html>

<head>
<meta http-equiv=Content-Type content="text/html; charset=utf-8">
<meta name=Generator content="Microsoft Word 15 (filtered)">
<style>
<!--
 /* Font Definitions */
 @font-face
	{{font-family:"Cambria Math";
	panose-1:2 4 5 3 5 4 6 3 2 4;}}
@font-face
	{{font-family:Calibri;
	panose-1:2 15 5 2 2 2 4 3 2 4;}}
@font-face
	{{font-family:Georgia;
	panose-1:2 4 5 2 5 4 5 2 3 3;}}
@font-face
	{{font-family:Cambria;
	panose-1:2 4 5 3 5 4 6 3 2 4;}}
@font-face
	{{font-family:Tahoma;
	panose-1:2 11 6 4 3 5 4 4 2 4;}}
@font-face
	{{font-family:"MS Sans Serif";
	panose-1:0 0 0 0 0 0 0 0 0 0;}}
@font-face
	{{font-family:TimesNewRomanPSMT;
	panose-1:0 0 0 0 0 0 0 0 0 0;}}
 /* Style Definitions */
 p.MsoNormal, li.MsoNormal, div.MsoNormal
	{{margin:0in;
	font-size:10.0pt;
	font-family:"Times New Roman",serif;}}
.MsoChpDefault
	{{font-size:10.0pt;}}
 /* Page Definitions */
 @page WordSection1
	{{size:595.3pt 841.9pt;
	margin:56.7pt 28.35pt 56.7pt 56.7pt;}}
div.WordSection1
	{{page:WordSection1;}}
 /* List Definitions */
 ol
	{{margin-bottom:0in;}}
ul
	{{margin-bottom:0in;}}
-->
</style>

</head>

<body lang=EN-US style='word-wrap:break-word'>

<div class=WordSection1>

<p class=MsoNormal align=center style='text-align:center;text-indent:35.45pt;
border:none'><b><span lang=RU style='font-size:12.0pt;color:black'>Дополнительное
соглашение №1</span></b></p>

<p class=MsoNormal align=center style='text-align:center;text-indent:35.45pt;
border:none'><b><span lang=RU style='font-size:12.0pt;color:black'>к договору от {timestamp} № {number}</span></b></p>

<p class=MsoNormal align=center style='text-align:center;text-indent:35.45pt;
border:none'><span lang=RU style='font-size:12.0pt;color:black'>&nbsp;</span></p>

<p class=MsoNormal style='display: flex; text-align:justify;border:none'><span lang=RU
style='font-size:12.0pt;color:black'>{place}</span><span style=" display: inline-block; margin-left: auto;">{timestamp}</span></p>

<p class=MsoNormal style='text-align:justify;text-indent:35.45pt;border:none'><span
lang=RU style='font-size:12.0pt;color:black'>&nbsp;</span></p>

<p class=MsoNormal style='text-align:justify;text-indent:35.45pt;border:none'><b><span
lang=RU style='font-size:12.0pt;color:black'>{party_1}</span></b><span lang=RU
style='font-size:12.0pt;color:black'>, в лице {genitive(position_1)} {genitive(name_1)}, действующего на основании ____________, именуемое в дальнейшем «<b>Заказчик</b>», с одной стороны, и </span></p>

<p class=MsoNormal style='text-align:justify;text-indent:35.45pt;border:none'><b><span
lang=RU style='font-size:12.0pt;color:black'>{party_2}</span></b><span lang=RU style='font-size:
12.0pt;color:black'>, именуемое в дальнейшем<b> «Исполнитель», </b>в лице {genitive(position_2)}
{genitive(party_2)}, действующего на основании ____________, с другой
стороны, вместе именуемые «Стороны» и каждый в отдельности «Сторона»,<b> </b>заключили
настоящее Дополнительное соглашение к к договору от {timestamp} № {number} (далее – Договор) о нижеследующем:</span></p>

<p class=MsoNormal style='margin-left:0in;text-align:justify;text-indent:35.45pt;
border:none'><span lang=RU style='font-size:12.0pt;color:black'>1.<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><span lang=RU style='font-size:12.0pt;color:black'>Стороны приняли решение о внесении следующих изменений в условия Договора:</span></p>""" + '\n\n'.join([f"<p class=MsoNormal style='margin-left:0in;text-align:justify;text-indent:35.45pt; border:none'><span lang=RU style='font-size:12.0pt'>{p}</span></p>" for p in gemma_generation.split('\n')]) + f"""<p class=MsoNormal style='margin-left:0in;text-align:justify;text-indent:35.45pt;
border:none'><span lang=RU style='font-size:12.0pt;color:black'>2.<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><span lang=RU style='font-size:12.0pt;color:black'>Настоящее
Дополнительное соглашение № 1 является неотъемлемой частью Договора.</span></p>

<p class=MsoNormal style='margin-left:0in;text-align:justify;text-indent:35.45pt;
border:none'><span lang=RU style='font-size:12.0pt;color:black'>3.<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><span lang=RU style='font-size:12.0pt;color:black'>Настоящее
Дополнительное соглашение № 1 считается заключенным и вступает в силу с даты его подписания Сторонами и действует до полного исполнения
Сторонами своих обязательств.</span></p>

<p class=MsoNormal style='margin-left:0in;text-align:justify;text-indent:35.45pt;
border:none'><span lang=RU style='font-size:12.0pt;color:black'>4.<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><span lang=RU style='font-size:12.0pt;color:black'>По всем иным
вопросам, не затронутым в настоящем Дополнительном соглашении № 1, Стороны
руководствуются условиями Договора и действующим законодательством РФ.</span></p>

<p class=MsoNormal style='margin-left:0in;text-align:justify;text-indent:35.45pt;
border:none'><span lang=RU style='font-size:12.0pt;color:black'>5.<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span><span lang=RU style='font-size:12.0pt;color:black'>Подписи
Сторон:</span></p>

<p class=MsoNormal style='margin-left:35.45pt;text-align:justify;border:none'><span
lang=RU style='font-size:12.0pt;color:black'>&nbsp;</span></p>

<div align=center>

<table class=1 border=0 cellspacing=0 cellpadding=0 width=624 style='border-collapse:
 collapse'>
 <tr style='height:176.95pt'>
  <td width=338 valign=top style='width:253.4pt;padding:0in 4.2pt 0in 4.2pt;
  height:176.95pt'>
  <p class=MsoNormal style='border:none'><b><span lang=RU style='font-size:
  12.0pt;color:black'>Заказчик:</span></b></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>{party_1}<b> </b></span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><b><span lang=RU
  style='font-size:12.0pt;color:black'>{position_1.capitalize()}</span></b></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><b><span lang=RU
  style='font-size:12.0pt;color:black'>_____________/ {name_1.split(' ')[0]}</span></b></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><b><span lang=RU
  style='font-size:12.0pt;color:black'>М.П.</span></b></p>
  <p class=MsoNormal style='margin-right:24.1pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  </td>
  <td width=286 valign=top style='width:214.7pt;padding:0in 4.2pt 0in 4.2pt;
  height:176.95pt'>
  <p class=MsoNormal style='margin-right:24.1pt;border:none'><b><span lang=RU
  style='font-size:12.0pt;color:black'>Поставщик</span></b><span lang=RU
  style='font-size:12.0pt;color:black'>:</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>{party_2}</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><b><span lang=RU
  style='font-size:12.0pt;color:black'>{position_2.capitalize()}</span></b></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  <p class=MsoNormal style='margin-right:10.0pt;border:none'><b><span lang=RU
  style='font-size:12.0pt;color:black'> _____________/ {name_2.split(' ')[0]}</span></b></p>
  <p class=MsoNormal style='margin-right:24.1pt;border:none'><b><span lang=RU
  style='font-size:12.0pt;color:black'>  М.П.</span></b></p>
  <p class=MsoNormal style='margin-right:24.1pt;border:none'><span lang=RU
  style='font-size:12.0pt;color:black'>&nbsp;</span></p>
  </td>
 </tr>
</table>

</div>

<p class=MsoNormal style='border:none'><span lang=RU style='font-size:12.0pt;
color:black'>&nbsp;</span></p>

</div>

</body>

</html>
"""

ERROR:root:'NoneType' object has no attribute 'word': Общество с ограниченной ответственностью «Автоэксклюзив»


In [164]:
# Save the string to an HTML file
with open("sample.html", "w") as file:
    file.write(template)